In [1]:
import pandas
import datetime
import numpy
from scipy.optimize import curve_fit
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.colors import Normalize
from matplotlib import ticker
from matplotlib import rcParams
%matplotlib inline

In [2]:
# setup some cutoff values we'll use in the analysis
velCutoffUpper = 2000.
velCutoffLower = 0.
numPointsCutoffMLTMLAT = 250
mlatCutOffUpper = 70.
probOccCutoff = 0.2

In [3]:
velGmagDF = pandas.read_csv("../data/processed-vels-geomag.txt", sep=' ')
velGmagDF = velGmagDF.drop('Unnamed: 0', axis=1)
# Discard unwanted values
# We'll only consider those velocities 
# which lie between 0 and 2500 m/s
# and located below 70 MLAT
velGmagDF = velGmagDF[ (velGmagDF["vSaps"] > velCutoffLower) \
                        & (velGmagDF["vSaps"] < velCutoffUpper)\
                       ].reset_index(drop=True)
velGmagDF = velGmagDF[ velGmagDF["MLAT"] < mlatCutOffUpper ].reset_index(drop=True)
# Now filter out velocities which have very few rate of occ.
# We calculat the prob and remove every measurement below 0.2 prob of occ.
mlatMLTDstCountDF = velGmagDF.groupby( ["MLAT", "normMLT", "dst_bin"] )["vSaps"].count().reset_index()
mlatMLTDstCountDF.columns = [ "MLAT", "normMLT", "dst_bin", "count" ]
dstMaxCntDF = mlatMLTDstCountDF.groupby( ["dst_bin"] )["count"].max().reset_index()
dstMaxCntDF.columns = [ "dst_bin", "maxCntDst" ]
mlatMLTDstCountDF = pandas.merge( mlatMLTDstCountDF, dstMaxCntDF, on=[ "dst_bin" ] )
mlatMLTDstCountDF["probOcc"] = mlatMLTDstCountDF["count"]/mlatMLTDstCountDF["maxCntDst"]
mlatMLTDstCountDF = mlatMLTDstCountDF[ mlatMLTDstCountDF["probOcc"] > probOccCutoff ].reset_index(drop=True)
# Filter out MLATs and MLTs (at the Dst bins)
# where number of measurements is low. We do
# this by merging the mlatMLTDstCountDF with velDF.
velGmagDF = pandas.merge( velGmagDF,\
                         mlatMLTDstCountDF,\
                         on=[ "MLAT", "normMLT", "dst_bin" ] )
velGmagDF = velGmagDF[ [ "normMLT", "MLAT", "vSaps",\
                        "azim", "dst_bin", "dst_index", "count", "maxCntDst" ] ]
# Divide the velocities into bins
velBins = [ v for v in range(0,int(velCutoffUpper)+100,100) ]
velGmagDF = pandas.concat( [ velGmagDF, \
                    pandas.cut( velGmagDF["vSaps"], \
                               bins=velBins ) ], axis=1 )
velGmagDF.columns = [ "normMLT", "MLAT", "vSaps",\
                        "azim", "dst_bin", "dst_index", "count",\
                         "maxCntDst", "vel_bin" ]
# velGmagDF.head()
# Get a DF with mean Dst in each bin
dstMeanDF = velGmagDF.groupby( ["dst_bin"] ).mean()["dst_index"].astype(int).reset_index()
dstMeanDF.columns = [ "dst_bin", "dst_mean" ]
velGmagDF = pandas.merge( velGmagDF, dstMeanDF, on=["dst_bin"] )
velGmagDF = velGmagDF.sort( ["dst_mean", "vSaps"], ascending=[False,True] ).reset_index(drop=True)
velCatOrderd = ['(0, 100]', '(100, 200]', '(200, 300]', '(300, 400]', '(400, 500]',\
          '(500, 600]', '(600, 700]', '(700, 800]',\
          '(800, 900]', '(900, 1000]', '(1000, 1100]', '(1100, 1200]',\
          '(1200, 1300]', '(1300, 1400]', '(1400, 1500]',\
          '(1500, 1600]', '(1600, 1700]', '(1700, 1800]',\
          '(1800, 1900]', '(1900, 2000]' ]

velGmagDF['vel_bin'] = pandas.Categorical(
    velGmagDF['vel_bin'], 
    categories=velCatOrderd, 
    ordered=True
)
# We need only a few cols
velGmagDF = velGmagDF[ [ "normMLT", "MLAT", "vSaps", "azim", "dst_index",\
                        "vel_bin", "dst_bin", "dst_mean" ] ]
velGmagDF.head()

,normMLT,MLAT,vSaps,azim,dst_index,vel_bin,dst_bin,dst_mean
0,1,58.5,164.32,-19.01,-7,"(100, 200]","(-10, 10]",-3
1,1,57.0,171.45,-16.11,-1,"(100, 200]","(-10, 10]",-3
2,1,59.0,174.58,-14.80,-7,"(100, 200]","(-10, 10]",-3
3,1,59.0,176.98,-6.09,-7,"(100, 200]","(-10, 10]",-3
4,2,59.5,178.11,-6.20,3,"(100, 200]","(-10, 10]",-3


In [4]:
# # Loop through each dst_bin, mlat and mlt....calculate the KDEs
# allDstBins = velGmagDF["dst_bin"].unique()
# allNormMLts = velGmagDF["normMLT"].unique()
# allMlats = velGmagDF["MLAT"].unique()
# # We'll loop through and calculate KDEs at each MLAT, MLT and Dst_bin
# # Store them in a DF
# velsKdeDF = pandas.DataFrame(columns=["dst_bin", "vel_arr", "pdf", "normMLT", "MLAT", "dst_mean"])
# dstBinArr = []
# dstMeanArr = []
# normMLTArr = []
# mlatArr = []
# velArr = []
# pdfArr = []
# # Define a inpVel arr to calculate kde
# inpVelsArr = numpy.arange(0.,2100.,100.)
# for cDst in allDstBins:
#     for cNmlt in allNormMLts:
#         for cMlat in allMlats:
#             currVelDF = velGmagDF[ (velGmagDF["dst_bin"] == cDst) &\
#                    (velGmagDF["MLAT"] == cMlat) &\
#                    (velGmagDF["normMLT"] == cNmlt)].reset_index(drop=True)
#             if currVelDF.shape[0] == 0:
#                 continue
#             velKernel = stats.gaussian_kde( currVelDF["vSaps"] )
#             pdfKernArr = velKernel.pdf( inpVelsArr )
#             dstBinArr.append( cDst )
#             normMLTArr.append( cNmlt )
#             mlatArr.append( cMlat )
#             velArr.append( inpVelsArr )
#             pdfArr.append( pdfKernArr )
#             dstMeanArr.append( currVelDF["dst_mean"].unique()[0] )
            
# velsKdeDF["dst_bin"] = dstBinArr
# velsKdeDF["vel_arr"] = velArr
# velsKdeDF["pdf"] = pdfArr
# velsKdeDF["normMLT"] = normMLTArr
# velsKdeDF["MLAT"] = mlatArr
# velsKdeDF["dst_mean"] = dstMeanArr
# velsKdeDF.head()

In [5]:
# Loop through each dst_bin, mlat and mlt....calculate the KDEs
allDstBins = velGmagDF["dst_bin"].unique()
allNormMLts = velGmagDF["normMLT"].unique()
allMlats = velGmagDF["MLAT"].unique()
# We'll loop through and calculate KDEs at each MLAT, MLT and Dst_bin
# Store them in a DF
velsKdeDF = pandas.DataFrame(columns=["dst_bin", "vel", "pdf", "normMLT", "MLAT", "dst_mean"])
dstBinArr = []
dstMeanArr = []
normMLTArr = []
mlatArr = []
velArr = []
pdfArr = []
# Define a inpVel arr to calculate kde
inpVelsArr = numpy.arange(0.,2100.,100.)
for cDst in allDstBins:
    for cNmlt in allNormMLts:
        for cMlat in allMlats:
            currVelDF = velGmagDF[ (velGmagDF["dst_bin"] == cDst) &\
                   (velGmagDF["MLAT"] == cMlat) &\
                   (velGmagDF["normMLT"] == cNmlt)].reset_index(drop=True)
            if currVelDF.shape[0] == 0:
                continue
            velKernel = stats.gaussian_kde( currVelDF["vSaps"] )
            pdfKernArr = velKernel.pdf( inpVelsArr )
            for npd in range(pdfKernArr.shape[0]):
                dstBinArr.append( cDst )
                normMLTArr.append( cNmlt )
                mlatArr.append( cMlat )
                velArr.append( inpVelsArr[npd] )
                pdfArr.append( pdfKernArr[npd] )
                dstMeanArr.append( currVelDF["dst_mean"].unique()[0] )
            
velsKdeDF["dst_bin"] = dstBinArr
velsKdeDF["vel"] = velArr
velsKdeDF["pdf"] = pdfArr
velsKdeDF["normMLT"] = normMLTArr
velsKdeDF["MLAT"] = mlatArr
velsKdeDF["dst_mean"] = dstMeanArr
velsKdeDF.head()

,dst_bin,vel,pdf,normMLT,MLAT,dst_mean
0,"(-10, 10]",0,0.000007,1,58.5,-3
1,"(-10, 10]",100,0.000156,1,58.5,-3
2,"(-10, 10]",200,0.000980,1,58.5,-3
3,"(-10, 10]",300,0.002113,1,58.5,-3
4,"(-10, 10]",400,0.002154,1,58.5,-3


In [6]:
# Define the fitting function
# We know that the velocities are 
# exhibiting a skew normal distribution
def fit_vel_pdf((mlt,mlat, dst, inpVel), a_ascmlt, b_ascmlt, a_bscmlt, b_bscmlt, a_cscmlt, b_cscmlt,\
               a_asclat, b_asclat, a_bsclat, b_bsclat,\
               a_ashmlt, b_ashmlt, a_bshmlt, b_bshmlt,\
               a_ashlat, b_ashlat, a_bshlat, b_bshlat,\
               a_alcmlt, b_alcmlt, a_blcmlt, b_blcmlt,\
               a_alclat, b_alclat, a_blclat, b_blclat):
    
    # model scale parameters
    # mlt
    a_scale_mlt = a_ascmlt + b_ascmlt * dst
    b_scale_mlt = a_bscmlt + b_bscmlt * dst
    c_scale_mlt = a_cscmlt + b_cscmlt * dst
    # mlat
    a_scale_mlat = a_asclat + b_asclat * dst
    b_scale_mlat = a_bsclat + b_bsclat * dst
    # func
    scale = ( a_scale_mlt + b_scale_mlt*(mlt) + c_scale_mlt*(mlt**2) ) * ( a_scale_mlat + b_scale_mlat*(mlat) )
    
    # model shape parameters
    # mlt
    a_shape_mlt = a_ashmlt + b_ashmlt * dst
    b_shape_mlt = a_bshmlt + b_bshmlt * dst
    # mlat
    a_shape_mlat = a_ashlat + b_ashlat * dst
    b_shape_mlat = a_bshlat + b_bshlat * dst
    # func
    shape = ( a_shape_mlt + b_shape_mlt*(mlt) ) * ( a_shape_mlat + b_shape_mlat*(mlat) )
    
    # model loc parameters
    # mlt
    a_loc_mlt = a_alcmlt + b_alcmlt * dst
    b_loc_mlt = a_blcmlt + b_blcmlt * dst
    # malt
    a_loc_mlat = a_alclat + b_alclat * dst
    b_loc_mlat = a_blclat + b_blclat * dst
    # func
    loc = ( a_loc_mlt + b_loc_mlt*(mlt) ) * ( a_loc_mlat*numpy.exp(b_loc_mlat*mlat) )
    
    # we need to adjust the skewnormal distribution
    # to account fot loc and scale parameters
    inpData = (inpVel - loc)/scale
    skNrml = 2*stats.norm.pdf(inpData)*stats.norm.cdf(shape*inpData)
    return skNrml.ravel()

initGuess = ( -1e+4, -1e+4, 1e3, 1e3, -10, -10,\
             100, 100, 10, 10,\
             1, 1, 10, 10,\
            -1, -1, 100, 100,\
            -100, -100, 100, 100,\
            4, 4, 0.1, 0.1)
popt2, pcov2 = curve_fit(fit_vel_pdf, (velsKdeDF['MLAT'].T,velsKdeDF['normMLT'].T,velsKdeDF['dst_mean'].T,\
                                       velsKdeDF['vel'].T), velsKdeDF['pdf'], p0=initGuess)
print popt2

[  1.81714958e+07   1.94085843e+04   1.91534371e+05   2.59418347e+03
   5.17672046e+02   3.89966980e+01  -2.17885389e+05  -2.64382318e+03
   7.34789569e+04   1.44952571e+04  -2.07642597e+01  -1.93232204e+01
  -1.32682196e+03  -8.73024054e+02   8.60384308e+01   1.29296723e+02
  -6.19101649e+01  -9.63429389e+02  -8.29408784e+06  -1.20723521e+05
  -1.74979097e+05  -1.20631777e+03  -8.83219586e+03  -3.77088789e+01
   2.73092385e+02   1.97703375e+00]


/usr/local/lib/python2.7/site-packages/scipy/optimize/minpack.py:715: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


In [11]:
def pred_vel_pdf(mlt,mlat, dst, inpVel):
    # Setup the fit results
    (a_ascmlt, b_ascmlt, a_bscmlt, b_bscmlt, a_cscmlt, b_cscmlt,\
               a_asclat, b_asclat, a_bsclat, b_bsclat,\
               a_ashmlt, b_ashmlt, a_bshmlt, b_bshmlt,\
               a_ashlat, b_ashlat, a_bshlat, b_bshlat,\
               a_alcmlt, b_alcmlt, a_blcmlt, b_blcmlt,\
               a_alclat, b_alclat, a_blclat, b_blclat) = (18171495.867044359,\
                19408.633031266352, 191534.37216077963, 2594.1831601616668,\
                517.67203713917093, 38.996694601356424, -217885.3884678616,\
                -2643.8231942847328, 73478.95662138937, 14495.257120556531,\
                -20.764254531577805, -19.323221763535734, -1326.8219604024803,\
                -873.02405437678726, 86.038430029207404, 129.29672221603241,\
                -61.910163921387628, -963.42938805378196, -8294087.8389505409,\
                -120723.52077821903, -174979.09653228571, -1206.3177722632845,\
                -8832.1958590658469, -37.708878919823192, 273.09238485621705, 1.9770337471672017)
    # model scale parameters
    # mlt
    a_scale_mlt = a_ascmlt + b_ascmlt * dst
    b_scale_mlt = a_bscmlt + b_bscmlt * dst
    c_scale_mlt = a_cscmlt + b_cscmlt * dst
    # mlat
    a_scale_mlat = a_asclat + b_asclat * dst
    b_scale_mlat = a_bsclat + b_bsclat * dst
    # func
    scale = ( a_scale_mlt + b_scale_mlt*(mlt) + c_scale_mlt*(mlt**2) ) * ( a_scale_mlat + b_scale_mlat*(mlat) )
    
    # model shape parameters
    # mlt
    a_shape_mlt = a_ashmlt + b_ashmlt * dst
    b_shape_mlt = a_bshmlt + b_bshmlt * dst
    # mlat
    a_shape_mlat = a_ashlat + b_ashlat * dst
    b_shape_mlat = a_bshlat + b_bshlat * dst
    # func
    shape = ( a_shape_mlt + b_shape_mlt*(mlt) ) * ( a_shape_mlat + b_shape_mlat*(mlat) )
    
    # model loc parameters
    # mlt
    a_loc_mlt = a_alcmlt + b_alcmlt * dst
    b_loc_mlt = a_blcmlt + b_blcmlt * dst
    # malt
    a_loc_mlat = a_alclat + b_alclat * dst
    b_loc_mlat = a_blclat + b_blclat * dst
    # func
    loc = ( a_loc_mlt + b_loc_mlt*(mlt) ) * ( a_loc_mlat*numpy.exp(b_loc_mlat*mlat) )
    
    print mlt,mlat, dst, inpVel, loc, shape, scale
    
    # we need to adjust the skewnormal distribution
    # to account fot loc and scale parameters
    inpData = (inpVel - loc)/scale
    pdfVal = 2*stats.norm.pdf(inpData)*stats.norm.cdf(shape*inpData)
    
    return pdfVal



In [12]:
predPdfArr = []
for index, row in velsKdeDF.iterrows():
    predPdfArr.append( pred_vel_pdf( row["normMLT"],row["MLAT"], row["dst_mean"], row["vel"] ) )
velsKdeDF["pred_pdf"] = predPdfArr

1.0 58.5 -3 0.0 inf 219570577.569 2.82630663931e+13
1.0 58.5 -3 100.0 inf 219570577.569 2.82630663931e+13
1.0 58.5 -3 200.0 inf 219570577.569 2.82630663931e+13
1.0 58.5 -3 300.0 inf 219570577.569 2.82630663931e+13
1.0 58.5 -3 400.0 inf 219570577.569 2.82630663931e+13
1.0 58.5 -3 500.0 inf 219570577.569 2.82630663931e+13
1.0 58.5 -3 600.0 inf 219570577.569 2.82630663931e+13
1.0 58.5 -3 700.0 inf 219570577.569 2.82630663931e+13
1.0 58.5 -3 800.0 inf 219570577.569 2.82630663931e+13
1.0 58.5 -3 900.0 inf 219570577.569 2.82630663931e+13
1.0 58.5 -3 1000.0 inf 219570577.569 2.82630663931e+13
1.0 58.5 -3 1100.0 inf 219570577.569 2.82630663931e+13
1.0 58.5 -3 1200.0 inf 219570577.569 2.82630663931e+13
1.0 58.5 -3 1300.0 inf 219570577.569 2.82630663931e+13
1.0 58.5 -3 1400.0 inf 219570577.569 2.82630663931e+13
1.0 58.5 -3 1500.0 inf 219570577.569 2.82630663931e+13
1.0 58.5 -3 1600.0 inf 219570577.569 2.82630663931e+13
1.0 58.5 -3 1700.0 inf 219570577.569 2.82630663931e+13
1.0 58.5 -3 1800.0 inf

In [9]:
velsKdeDF[ velsKdeDF["pred_pdf"] > 0. ].head()

,dst_bin,vel,pdf,normMLT,MLAT,dst_mean,pred_pdf


[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,